# [Module 2.1] Related 데이타 준비 (Prepare Related data file) 

이 노트북은 이전 노트북의 Target 데이터 셋에 대한 Related 데이터 셋을 만들게 됩니다. Target 데이터의 Key 필드 (예: date, item_id)에 해당하는 모두의 Related 값을 만들어 구성해야 합니다.

* 이 노트북에서는 입력 데이타를 Forecast에 입력이 될 포맷으로 학습 (Train) 과 검증셋(Validation)으로 나누는 작업을 합니다. **자세한 과정을 알고 싶으시면 0.LookAt_RawData.ipynb 노트북 을 실행 해주세요.** 

* 이 노트북은 약 3분 정도 소요 됩니다. 
    

In [1]:
import boto3
import os
import pandas as pd
import numpy as np

## Related 데이터 셋 작성

**원천 데이타가 일주일의 시작이 금요일 부터 시작이 되고, Amazon Forecast는 월요일 부터 시작이 되기에 인위적으로 원본 데이타를 미래로 3일 앞으로 이동 하였습니다.**

IMPORTANT: Shift three days in the future because a weekly interval starts from Monday in Amazon Forecast.This dataset starts from Friday for an weekly interval



In [2]:
data_dir = 'data'
features_file_name = 'features.csv'
feature_data = pd.read_csv(os.path.join(data_dir,features_file_name))
feature_df = feature_data.copy()
# Shift three days in the future 
feature_df.Date = pd.to_datetime(feature_df['Date']) + pd.DateOffset(days=3)

**Temperature, Fuel_Price를 제외하고 모든 다른 컬럼은 제외 합니다. (결측값이 존재하고 단순화의 이유임)**

In [3]:
feature_df = feature_df.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'], axis=1)
# Drop CPI, Unemployment because of missing values
# Drop isHoliday because isHoliday exists in the built-in data of AWS Forecast
feature_df = feature_df.drop(['CPI','Unemployment','IsHoliday'], axis=1)



In [4]:
feature_df = feature_df.rename(columns={'Store':'item_id','Date':'date'})
feature_df.reset_index(inplace=True)
feature_df = feature_df.drop('index', axis=1)
feature_df = feature_df.set_index('date')
feature_df.item_id = feature_df.item_id.astype(str)
feature_df.index = pd.to_datetime(feature_df.index, format = '%Y-%m-%d')
cols = ['Temperature','Fuel_Price','item_id']
feature_df = feature_df[cols]
feature_df.head()

,Temperature,Fuel_Price,item_id
date,,,
2010-02-08,42.31,2.572,1
2010-02-15,38.51,2.548,1
2010-02-22,39.93,2.514,1
2010-03-01,46.63,2.561,1
2010-03-08,46.50,2.625,1


## Related 데이타 셋을 CSV 로 저장

- 훈련 데이터는 2010.02.08 ~ 2012.07.23 기간
- 검증 데이터는 2010.07.30 ~ 2012.10.22 기간

```end_val_date = '2012-10-29' ``` 이 의미는 Related 데이타 셋은 훈련 기간 및 앞으로 미래에 예측할 기간까지 모두 데이타를 넣는다는 의미 입니다.
이 부분은 다음과 같은 의미가 있습니다.

- 실제 포케스트 서비스가 서비스를 시작하면 "미래의 예측 기간 (ForecastHorizon 이라 칭함)"의 특정 데이타는 알수가 없습니다. 예를 들어서 Temperature는 미래의 날씨에 해당하는 온도이기에 사실상 알 수가 없습니다.**실제로 미래의 예측 기간에 값을 넣는다는 의미는 " '이 값을 넣으면 이렇게 예측이 될거야' 라는 What-If 예측이라고 봐야 합니다.**
- 예를 들어서,12월의 크리스마스 연휴에 대해서 특정 아이템은 (예: item_id = 1) Promotion을 1로 특정 아이템(예: item_id = 2) 는 Promotion을 2로 Related 데이타 셋을 구성하여 사용한다면, 이에 따른 What-If 개념으로 예측을 하게 됩니다.

In [5]:
end_val_date = '2012-10-29' 

In [6]:
related_feature_df = feature_df[feature_df.index < end_val_date]

In [7]:
# With the data in a great state, save it off as a CSV
related_time_series_filename = "related_time_series.csv"
related_time_series_path = data_dir + "/" + related_time_series_filename
related_feature_df.to_csv(related_time_series_path, header=False)

In [8]:
%store related_time_series_filename
%store related_time_series_path
%store related_feature_df


Stored 'related_time_series_filename' (str)
Stored 'related_time_series_path' (str)
Stored 'related_feature_df' (DataFrame)
